In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tqdm import tqdm


In [2]:

# Load MovieLens Small dataset
data = pd.read_csv("../../datasets/ml-latest-small/ratings.csv", sep=",")
#data = pd.read_csv("../../datasets/ml-1m/ratings.csv", sep=",", names=["userId","movieId","rating","timestamp"])

#data = pd.read_csv("ratings.csv", sep=",")
#prefs = pd.read_csv("drive/MyDrive/PycharmProjects/datasets/ml-latest-small/ratings.csv", sep=",")

data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
data.shape

(100836, 4)

In [4]:
# Map user and movie IDs to unique indices
user_ids = data['userId'].unique()
movie_ids = data['movieId'].unique()

user_mapping = {user_id: idx for idx, user_id in enumerate(user_ids)}
movie_mapping = {movie_id: idx for idx, movie_id in enumerate(movie_ids)}

data['userId'] = data['userId'].map(user_mapping)
data['movieId'] = data['movieId'].map(movie_mapping)

In [5]:
class MovieLensDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe[['userId', 'movieId']].values
        self.ratings = dataframe['rating'].values.astype(np.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.LongTensor(self.data[idx]), torch.FloatTensor([self.ratings[idx]])


In [6]:
# Define the matrix factorization model
class MatrixFactorization(nn.Module):
    def __init__(self, num_users, num_movies, embedding_size=20):
        super(MatrixFactorization, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.movie_embedding = nn.Embedding(num_movies, embedding_size)
        self.linear = nn.Linear(2*embedding_size, 1)

    def forward(self, X):
        user_embedding = self.user_embedding(X[:,0])
        movie_embedding = self.movie_embedding(X[:,1])
        #print("***********")
        #print(user_embedding)
        #print("***********")
        #print(movie_embedding)
        #print("***********")
        #a = torch.cat((user_embedding, movie_embedding), dim=1)
        #prediction = self.linear(a)
        #prediction = torch.sum(user_embedding + movie_embedding, dim=1)
        prediction = torch.sum(user_embedding * movie_embedding, dim=1)
        #print(prediction)
        #print("***********")

        return prediction

In [16]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.1)

# Initialize dataset and data loaders
train_dataset = MovieLensDataset(train_data)
test_dataset = MovieLensDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [13]:
a = torch.Tensor([[-0.4266, -0.4373, -0.4427, -0.7173,  0.1183],
        [ 0.5720,  1.3187, -1.5295, -1.7553,  0.9981]])
b = torch.Tensor([[-0.1013,  0.8980, -0.3187,  0.5583, -1.4790],
        [ 0.1501,  0.8112, -0.4382,  0.5182,  0.3780]])
torch.sum(a * b, dim=1)

tensor([-0.7838,  1.2935])

In [17]:
# Initialize the model and optimizer
num_users = len(user_ids)
num_movies = len(movie_ids)
model = MatrixFactorization(num_users, num_movies, embedding_size=10)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device {device}.")
model = model.to(device)

criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.03, momentum=0.9)

# Training loop
num_epochs = 20

import time
start = time.time()

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    absolute_errors = []

    for batch_idx, (X, y) in enumerate(train_loader):
        optimizer.zero_grad()
        X = X.to(device)
        y = y.to(device)
        predictions = model(X).squeeze()
        loss = criterion(predictions, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        absolute_errors.extend(torch.abs(predictions - y).tolist()[0])
        


    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")
    print(f"Train MAE: {np.mean(absolute_errors)}")

end = time.time()
print("Elapsed time:", end - start)


Using device cpu.


/Users/tevfikaytekin/miniforge3/envs/pytorch/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([16, 1])) that is different to the input size (torch.Size([16])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/20, Loss: 9.16546638941521
Train MAE: 2.337343557430392
Epoch 2/20, Loss: 2.545878489673852
Train MAE: 1.1694628261269446
Epoch 3/20, Loss: 1.7499732121783185
Train MAE: 0.9953377482340748
Epoch 4/20, Loss: 1.444319852531923
Train MAE: 0.9347572538805324
Epoch 5/20, Loss: 1.288792024331091
Train MAE: 0.9081613919966984
Epoch 6/20, Loss: 1.2087867896536257
Train MAE: 0.8830881842536187
Epoch 7/20, Loss: 1.1662138665360313
Train MAE: 0.8637266402764637
Epoch 8/20, Loss: 1.1414692258706634
Train MAE: 0.8678001060455313
Epoch 9/20, Loss: 1.1289233248260486
Train MAE: 0.8448741710541194
Epoch 10/20, Loss: 1.1205149074787817
Train MAE: 0.8482443097117127
Epoch 11/20, Loss: 1.1158572624769845
Train MAE: 0.8514452156175994
Epoch 12/20, Loss: 1.1132212616240524
Train MAE: 0.8362070211433977
Epoch 13/20, Loss: 1.1114514961928046
Train MAE: 0.845196925424176
Epoch 14/20, Loss: 1.1100219005485013
Train MAE: 0.8347195634416111
Epoch 15/20, Loss: 1.109731382478885
Train MAE: 0.84260395873858

In [18]:
# Evaluation on the test set
model.eval()
test_loss = 0.0
absolute_errors = []
with torch.no_grad():
    for batch_idx, (X, y) in enumerate(test_loader):
        predictions = model(X).squeeze()
        loss = criterion(predictions, y)
        test_loss += loss.item()
        absolute_errors.extend(torch.abs(predictions - y).tolist()[0])

print(f"Test Loss: {test_loss / len(test_loader)}")
print(f"Test MAE: {np.mean(absolute_errors)}")

Test Loss: 1.887587298823612
Test MAE: 0.9501855440647627


/Users/tevfikaytekin/miniforge3/envs/pytorch/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4, 1])) that is different to the input size (torch.Size([4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
